In [1]:
import datetime
import random
import os

In [ ]:
# --- 설정 값 ---
output_filename = "generated_insert_data_for_update.sql"
num_records_to_insert = 1000
num_env_logs_multiplier = 7 # report 1개당 생성할 환경 로그 개수 (총 1000*7 = 7000개 예상)
batch_size = 100 # 한 번의 INSERT 문에 포함될 행 수

In [ ]:
# --- 필수 FK 및 NOT NULL 값 ---
baby_id_to_use = 6
device_id_to_use = 6
usage_log_id_to_use = 29590

In [ ]:
# --- 고정된 Prediction log 날짜 ---
prediction_fixed_date = datetime.date(2025, 6, 17)

In [ ]:
# --- Prediction log 시간 타임라인 생성 기준 시간 (2025-06-17 날짜 사용) ---
# 2024-09-22 타임라인 패턴을 모방하기 위한 시작 시간 및 간격 설정
# 예시: 하루 시작 시간, 잠 길이 범위, 깨어있는 시간 범위
prediction_timeline_start_of_day = datetime.datetime.combine(prediction_fixed_date, datetime.time(7, 0, 0)) # 2025-06-17 07:00 시작 예시
predicted_sleep_duration_range = (30, 120) # 예측 잠 길이: 30분 ~ 2시간
predicted_awake_interval_range = (60, 240) # 예측 깨어있는 시간: 1시간 ~ 4시간

In [ ]:
# --- 예상 시작 ID 주석 ---
starting_ids = {
    "device_environment_logs": 29590,
    "report": 29599,
    "sleep_prediction_log": 255,
}

In [ ]:
# --- 파일 열기 ---
try:
    with open(output_filename, "w") as f:
        # SQL 파일 헤더
        f.write("-- Generated SQL INSERT statements for later update (Many NULLs, Prediction times generated)\n")
        f.write(f"-- Generation Date: {datetime.datetime.now()}\n")
        f.write(f"-- For baby_id/device_id: {baby_id_to_use}, usage_log_id: {usage_log_id_to_use}\n")
        f.write(f"-- Prediction date fixed to: {prediction_fixed_date}\n")
        f.write("-- report.start_time, end_time, duration, breaks_count, sleep_efficiency_percent: NULL\n")
        f.write("-- device_environment_logs.temperature, humidity, brightness, white_noise_level, recorded_at: NULL\n")
        f.write("-- sleep_prediction_log.rep_id, expected_duration: NULL\n")
        f.write("-- sleep_prediction_log.expected_start_at, expected_end_at, expected_duration: Generated based on a timeline on 2025-06-17\n")

        f.write("\n")
        f.write("SET FOREIGN_KEY_CHECKS=0;\n") # 임시로 FK 체크 해제 - 실행 후 다시 1로 설정
        f.write("SET UNIQUE_CHECKS=0;\n") # 임시로 UNIQUE 체크 해제 - 실행 후 다시 1로 설정
        f.write("SET SQL_MODE='NO_AUTO_VALUE_ON_ZERO';\n") # AUTO_INCREMENT=0 사용 가능 설정 (일부 시스템용, 없어도 무방)
        f.write("\n")


        # --- Report INSERT 문 생성 ---
        print(f"Generating {num_records_to_insert} Report INSERT statements...")
        # report_id의 시작 예상 주석
        f.write(f"-- Starting rep_id is expected to be around {starting_ids['report']}\n")
        report_values = []

        for i in range(num_records_to_insert):
            # report_id (AI), baby_id (NOT NULL, 지정 값), start_time (NULL), end_time (NULL),
            # duration (NULL), breaks_count (NULL), sleep_efficiency_percent (NULL)
            report_values.append(f"({baby_id_to_use}, NULL, NULL, NULL, NULL, NULL)")

            if len(report_values) == batch_size or i == num_records_to_insert - 1:
                f.write(f"INSERT INTO report (baby_id, start_time, end_time, duration, breaks_count, sleep_efficiency_percent) VALUES\n  ")
                f.write(",\n  ".join(report_values))
                f.write(";\n")
                report_values = []
        f.write("\n")

        # --- Device Environment Logs INSERT 문 생성 ---
        print(f"Generating ~{num_records_to_insert * num_env_logs_multiplier} Device Environment Logs INSERT statements...")
        # environment_log_id의 시작 예상 주석
        f.write(f"-- Starting environment_log_id is expected to be around {starting_ids['device_environment_logs']}\n")
        env_log_values = []
        total_env_logs = num_records_to_insert * num_env_logs_multiplier

        for i in range(total_env_logs):
            # usage_log_id (NOT NULL, 지정 값), device_id (NOT NULL, 지정 값), sleep_mode (NOT NULL, day 고정)
            # temperature (NULL), humidity (NULL), brightness (NULL), white_noise_level (NULL), recorded_at (NULL) -> 고정값으로 수정
            # sleep_mode = random.choice(['day', 'night'])
            env_log_values.append(f"({usage_log_id_to_use}, {device_id_to_use}, 'day', 21.2, 68.6, 17.1, 38, NULL)")

            if len(env_log_values) == batch_size or i == total_env_logs - 1:
                f.write(f"INSERT INTO device_environment_logs (usage_log_id, device_id, sleep_mode, temperature, humidity, brightness, white_noise_level, recorded_at) VALUES\n  ")
                f.write(",\n  ".join(env_log_values))
                f.write(";\n")
                env_log_values = []
        f.write("\n")


        # --- Sleep Prediction Log INSERT 문 생성 ---
        print(f"Generating {num_records_to_insert} Sleep Prediction Log INSERT statements...")
        f.write(f"-- Starting prediction_id is expected to be around {starting_ids['sleep_prediction_log']}\n")
        f.write("-- rep_id needs to be matched later with report.rep_id based on INSERT ORDER\n")
        f.write(f"-- prediction_date is fixed to '{prediction_fixed_date.strftime('%Y-%m-%d')}'\n")
        f.write("-- expected_start_at, expected_end_at, expected_duration are generated based on a timeline on 2025-06-17\n") # 주석 수정

        prediction_values = []
        current_prediction_timeline_time = prediction_timeline_start_of_day

        for i in range(num_records_to_insert):
             # --- expected_start_at, expected_end_at 타임라인 생성 ---
             predicted_start_at_calc = current_prediction_timeline_time # 계산상의 시작 시간

             pred_duration_minutes_calc = random.randint(*predicted_sleep_duration_range) # 예측 길이 랜덤 생성 (분)
             predicted_end_at_calc = predicted_start_at_calc + datetime.timedelta(minutes=pred_duration_minutes_calc) # 계산상의 종료 시간

             # 다음 예측 시작 시간 계산
             awake_interval = random.randint(*predicted_awake_interval_range)
             next_prediction_timeline_time = predicted_end_at_calc + datetime.timedelta(minutes=awake_interval)

             # 타임라인 시간을 업데이트 (다음 루프에서 사용)
             current_prediction_timeline_time = next_prediction_timeline_time

             # --- INSERT VALUES에 사용할 시간: 날짜를 prediction_fixed_date로 강제 ---
             predicted_start_at_final = datetime.datetime.combine(prediction_fixed_date, predicted_start_at_calc.time())
             predicted_end_at_final = datetime.datetime.combine(prediction_fixed_date, predicted_end_at_calc.time())

             # --- expected_duration 계산 (분 단위 정수) ---
             # total_seconds()는 timedelta에서 총 초를 가져옴
             duration_timedelta = predicted_end_at_final - predicted_start_at_final
             expected_duration_minutes = int(duration_timedelta.total_seconds() / 60) # 분 단위 정수


             # --- INSERT VALUES 생성 ---
             # baby_id (NOT NULL), rep_id (NULL), prediction_date (지정 날짜),
             # expected_start_at (생성된 시간), expected_end_at (생성된 시간), expected_duration (계산된 값)

             prediction_values.append(
                 f"({baby_id_to_use}, NULL, '{prediction_fixed_date.strftime('%Y-%m-%d')}', "
                 f"'{predicted_start_at_final.strftime('%Y-%m-%d %H:%M:%S')}', "
                 f"'{predicted_end_at_final.strftime('%Y-%m-%d %H:%M:%S')}', {expected_duration_minutes})" # duration 값 포함
             )


            if len(prediction_values) == batch_size or i == num_records_to_insert - 1:
                f.write(f"INSERT INTO sleep_prediction_log (baby_id, rep_id, prediction_date, expected_start_at, expected_end_at, expected_duration) VALUES\n  ")
                f.write(",\n  ".join(prediction_values))
                f.write(";\n")
                prediction_values = []
        f.write("\n")

        # --- SQL 파일 꼬리말 ---
        f.write("SET SQL_MODE='';" + ("\n" if "SET SQL_MODE='NO_AUTO_VALUE_ON_ZERO';" in open(__file__).read() else "")) # SQL_MODE 복원 (스크립트 자체 확인)
        f.write("SET UNIQUE_CHECKS=1;\n") # UNIQUE 체크 다시 활성화
        f.write("SET FOREIGN_KEY_CHECKS=1;\n") # FK 체크 다시 활성화
        f.write("\n")
        f.write("-- End of generated SQL\n")

    print(f"\nSuccessfully generated SQL file: {output_filename}")
    print("\n--- NEXT STEPS ---")
    print("1. Run the generated SQL file in your database.")
    print(f"   Example command line: mysql -u YOUR_USER -p YOUR_DB_NAME < {output_filename}")
    print("\n2. After inserting, UPDATE the NULL columns in other tables with actual data.")
    print("   - report: start_time, end_time, duration, breaks_count, sleep_efficiency_percent")
    print("   - device_environment_logs: temperature, humidity, brightness, white_noise_level, recorded_at")
    # print("   - sleep_prediction_log: expected_duration") # Duration is now calculated
    print("\n3. Crucially, UPDATE sleep_prediction_log.rep_id to link predictions to reports.")
    print("   This needs to match based on insertion order (1st report <-> 1st prediction, etc.).")
    print("   You'll need to query for the newly inserted report IDs and prediction IDs to perform this update.")


except Exception as e:
    print(f"An error occurred: {e}")
    if os.path.exists(output_filename):
        try:
            os.remove(output_filename)
            print(f"Removed incomplete file: {output_filename}")
        except OSError as remove_err:
            print(f"Error removing incomplete file: {remove_err}")